In [51]:
import pandas as pd
%matplotlib notebook
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import numpy as np
import matplotlib.pyplot as plt

from bokeh.plotting import figure, output_file
from bokeh.charts import Bar, output_file, show
import bokeh.plotting as bplt
from bokeh.palettes import Dark2_5 as palette
from bokeh.models import Label, Title
import itertools  
from bokeh.layouts import gridplot,row,column

from bokeh.io import hplot, output_file, show
from bokeh.plotting import figure

from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
from bokeh.io import output_notebook
plt.style.use('ggplot')
output_notebook()


Loading BokehJS ...

In [45]:
directory = '/home/georgeha/repos/midas_exps/streaming/k-means/spark/'

def plot_producer(producer_dir,producer_filename,p_number,fig_count,txt):
    
    rates =pd.read_csv(directory + producer_dir + producer_filename)
    x_values = rates['Num_Messages'].tolist()
    y_values = rates['KB/sec'].tolist()
    
    f = plt.figure()
    plt.xlim(0,12500)
    plt.plot(x_values,y_values)
    plt.ylabel('KB/s')
    plt.xlabel('Msg number')
    plt.title("Producer " + str(p_number)+ "  throughput")
    text = 'Figure ' + str(fig_count) + ' : ' + txt
    f.text(.20, .02, text, ha='center')
    
    return (x_values,y_values)

In [46]:
directory = '/home/georgeha/repos/midas_exps/streaming/k-means/spark/'
def plot_scheduling_and_Total_Delay(spark_metrics_dir,fig_count,title):
    
    metrics = pd.read_csv(directory + spark_metrics_dir ,skipinitialspace=True)
    scheduling_delay = metrics['SchedulingDelay'].tolist()    
    TotalDelay = metrics['TotalDelay'].tolist()
    
    scheduling_delay = map(lambda x: x/1000, scheduling_delay)  # convert sec from milisec
    TotalDelay = map(lambda x: x/1000, TotalDelay)
    
    processing_time = []
    for i in xrange(len(scheduling_delay)):
        el = TotalDelay[i] - scheduling_delay[i]
        processing_time.append(el)

    x_values = range(0,metrics['SchedulingDelay'].count())
    # create a new plot with a title and axis labels
    p = figure(title="Figure " + str(fig_count) + ": Spark Scheduling delay " + title, x_axis_label='miniBatch Number', y_axis_label='Delay in sec')
    # add a line renderer with legend and line thickness
    p.line(x_values, scheduling_delay, legend="Scheduling Delay", line_width=2,line_color="red")
    p.line(x_values,TotalDelay,legend="Total Delay", line_width=2)
    p.line(x_values,processing_time,legend="Processing Time", line_width=2,line_color='green') # in seconds
    #throughput = (TotalDelay - scheduling_delay)/metrics['NumberRecords']
    bplt.show(p)
    return (x_values,scheduling_delay)

In [47]:
def plot_throughput_per_mini_batch(spark_metrics_dir,fig_count):
    
    metrics = pd.read_csv(directory + spark_metrics_dir ,skipinitialspace=True)
    scheduling_delay = metrics['SchedulingDelay']
    TotalDelay = metrics['TotalDelay']
    x_values = range(0,scheduling_delay.count())    
    throughput = metrics['NumberRecords']/60      # check That
    # create a new plot with a title and axis labels
    p = figure(title="Figure " + str(fig_count) + " : Spark throughput/mini batch  - batch=60sec", x_axis_label='miniBatch Number', y_axis_label='records/sec')

    # add a line renderer with legend and line thickness
    p.line(x_values, throughput.tolist(), legend="Throughput", line_width=2,line_color="green")

    # show the results
    bplt.show(p)
    return  (x_values,throughput.tolist())

In [61]:
def plot_throughput_and_Delays(spark_metrics_dir,fig_count):
    
    
    metrics = pd.read_csv(directory + spark_metrics_dir ,skipinitialspace=True)
    scheduling_delay = metrics['SchedulingDelay'].tolist()    
    TotalDelay = metrics['TotalDelay'].tolist()
    
    scheduling_delay = map(lambda x: x/1000, scheduling_delay)  # convert sec from milisec
    TotalDelay = map(lambda x: x/1000, TotalDelay)
    
    processing_time = []
    for i in xrange(len(scheduling_delay)):
        el = TotalDelay[i] - scheduling_delay[i]
        processing_time.append(el)

    x_values = range(0,metrics['SchedulingDelay'].count())
    # create a new plot with a title and axis labels
    p1 = figure(width=500, height=500,title="Figure " + str(fig_count) + ": Spark Scheduling delay " , x_axis_label='miniBatch Number', y_axis_label='Delay in sec')
    # add a line renderer with legend and line thickness
    p1.line(x_values, scheduling_delay, legend="Scheduling Delay", line_width=2,line_color="red")
    p1.line(x_values,TotalDelay,legend="Total Delay", line_width=2)
    p1.line(x_values,processing_time,legend="Processing Time", line_width=2,line_color='green') # in seconds
    
    
    scheduling_delay = metrics['SchedulingDelay']
    TotalDelay = metrics['TotalDelay']
    throughput = metrics['NumberRecords']/60      # check That
    # create a new plot with a title and axis labels
    p2 = figure(width=500, height=500,title="Figure " + str(fig_count) + " : Spark throughput/mini batch  - batch=60sec", x_axis_label='miniBatch Number', y_axis_label='records/sec')

    # add a line renderer with legend and line thickness
    p2.line(x_values, throughput.tolist(), legend="Throughput", line_width=2,line_color="green")
    
    # put all the plots in an HBox
    plots = hplot(p1, p2)

    # show the results
    bplt.show(plots)
    return
    

In [152]:
def plot_decays(spark_metrics_dir_1,title_1,spark_metrics_dir_2,title_2,fig_count):
    
    metrics = pd.read_csv(directory + spark_metrics_dir_1 ,skipinitialspace=True)
    scheduling_delay = metrics['SchedulingDelay'].tolist()    
    TotalDelay = metrics['TotalDelay'].tolist()
    
    scheduling_delay = map(lambda x: x/1000, scheduling_delay)  # convert sec from milisec
    TotalDelay = map(lambda x: x/1000, TotalDelay)
    
    processing_time = []
    for i in xrange(len(scheduling_delay)):
        el = TotalDelay[i] - scheduling_delay[i]
        processing_time.append(el)

    x_values = range(0,metrics['SchedulingDelay'].count())
    # create a new plot with a title and axis labels
    p1 = figure(title="Figure " + str(fig_count) + ": Spark Scheduling delay " + title_1, x_axis_label='miniBatch Number', y_axis_label='Delay in sec')
    p1.line(x_values, scheduling_delay, legend="Scheduling Delay", line_width=2,line_color="red")
    p1.line(x_values,TotalDelay,legend="Total Delay", line_width=2)
    p1.line(x_values,processing_time,legend="Processing Time", line_width=2,line_color='green') # in seconds
    ##### 2nd plot
    
    metrics = pd.read_csv(directory + spark_metrics_dir_2 ,skipinitialspace=True)
    scheduling_delay = metrics['SchedulingDelay'].tolist()    
    TotalDelay = metrics['TotalDelay'].tolist()
    
    scheduling_delay = map(lambda x: x/1000, scheduling_delay)  # convert sec from milisec
    TotalDelay = map(lambda x: x/1000, TotalDelay)
    
    processing_time = []
    for i in xrange(len(scheduling_delay)):
        el = TotalDelay[i] - scheduling_delay[i]
        processing_time.append(el)

    x_values = range(0,metrics['SchedulingDelay'].count())
    # create a new plot with a title and axis labels
    p2 = figure(title="Figure " + str(fig_count) + ": Spark Scheduling delay " + title_2, x_axis_label='miniBatch Number', y_axis_label='Delay in sec')
    p2.line(x_values, scheduling_delay, legend="Scheduling Delay", line_width=2,line_color="red")
    p2.line(x_values,TotalDelay,legend="Total Delay", line_width=2)
    p2.line(x_values,processing_time,legend="Processing Time", line_width=2,line_color='green') # in seconds
    
    # put all the plots in an HBox
    plots = hplot(p1, p2)

    # show the results
    bplt.show(plots)
    return
    
    
    
    
    
    

# Spark Streaming  on 1 wrangler node

In [48]:
directory = directory + '1-node/'

## 1-1-1

In [142]:
plot_throughput_and_Delays('1-1-1/spark-metrics-20171003-153347.csv',1)

/home/georgeha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:35: BokehDeprecationWarning: bokeh.io.hplot was deprecated in Bokeh 0.12.0; please use bokeh.models.layouts.Row instead


In [144]:
aa = pd.read_csv(directory+ '1-1-1/spark-metrics-20171003-153347.csv')
td111 = ((aa[' TotalDelay']/1000).sum())/60 #minutes
sd111 = (aa[' SchedulingDelay']/1000).sum()/60 #minutes
throughput111 = aa[' NumberRecords'].sum()/td111  # minutes
pt111 = td111 - sd111
print 'Case: 1-1-1'
print 'Total delay: ' + str(td111) + '  minutes'
print 'Processing time:  ' + str(pt111)  + '   minutes'
print 'Scheduling delay: ' + str(sd111) + '  minutes'
print 'Throughput : ' + str(throughput111) + ' records/minute'
print 'Throughput : ' + str(throughput111/60) + ' records/sec'

#TODO: ADD throughput

Case: 1-1-1
Total delay: 5.5197  minutes
Processing time:  5.51903333333   minutes
Scheduling delay: 0.000666666666667  minutes
Throughput : 4025.94343896 records/minute
Throughput : 67.0990573159 records/sec


## 8-1-1

In [33]:
#p811_1_1r = plot_producer('8-1-1-1r/producers/producer-1/','stdout-20171011-224339.csv',1,1,'5k-8-1-1')
#p811_2_1r = plot_producer('8-1-1-1r/producers/producer-2/','stdout-20171011-224339.csv',2,2,'5k-8-1-1')
#p811_3_1r = plot_producer('8-1-1-1r/producers/producer-3/','stdout-20171011-224339.csv',3,3,'5k-8-1-1')
#p811_4_1r = plot_producer('8-1-1-1r/producers/producer-4/','stdout-20171011-224339.csv',4,4,'5k-8-1-1')
#p811_5_1r = plot_producer('8-1-1-1r/producers/producer-5/','stdout-20171011-224339.csv',5,5,'5k-8-1-1')
#p811_6_1r = plot_producer('8-1-1-1r/producers/producer-6/','stdout-20171011-224339.csv',6,6,'5k-8-1-1')
#p811_7_1r = plot_producer('8-1-1-1r/producers/producer-7/','stdout-20171011-224339.csv',7,7,'5k-8-1-1')
#p811_8_1r = plot_producer('8-1-1-1r/producers/producer-8/','stdout-20171011-224339.csv',8,8,'5k-8-1-1')


In [65]:
p = figure(width = 500, height= 500, title="Figure " + str(7) + ":Produers throughput in case 8-1-1 ", x_axis_label='msg Number', y_axis_label='throughput in KB/s')

p.line(p811_1_1r[0], p811_1_1r[1], legend="p-1", line_width=2,line_color="red")
p.line(p811_2_1r[0], p811_2_1r[1], legend="p-2", line_width=2,line_color="blue")
p.line(p811_3_1r[0], p811_3_1r[1], legend="p-3", line_width=2,line_color="green")
p.line(p811_4_1r[0], p811_4_1r[1], legend="p-4", line_width=2,line_color="black")
p.line(p811_5_1r[0], p811_5_1r[1], legend="p-5", line_width=2,line_color="purple")
p.line(p811_6_1r[0], p811_6_1r[1], legend="p-6", line_width=2,line_color="yellow")
p.line(p811_7_1r[0], p811_7_1r[1], legend="p-7", line_width=2,line_color="pink")
p.line(p811_8_1r[0], p811_8_1r[1], legend="p-8", line_width=2,line_color="orange")


bplt.show(p)

In the experiment above I am utilizng one wrangler node to do measure producer's throughput and computation. I am running 8 concurent producers (in the above plot) and scale up to 48 <br>
 * Each producer is producing 2.5GB of data (floats)
 * Application: Spark Streaming k-means with 1 Consumer Node
 * Message size is 5K 3-D points
 ##### Observations: 
 * The throughput of each producer is different. The difference is up to 3MB/s
 * 1025 sec which is 20 minutes is the total time of data production
 * My hypothesis is that the broker can't handle 8 different requests from 8 different producers


##### consumer  - Spark Scheduling Delay -  Total Delay - Processing time

In [62]:
plot_throughput_and_Delays('8-1-1-1r/spark-metrics-20171011-224252.csv',3)

/home/georgeha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:35: BokehDeprecationWarning: bokeh.io.hplot was deprecated in Bokeh 0.12.0; please use bokeh.models.layouts.Row instead


In [141]:
aa = pd.read_csv(directory+ '8-1-1-1r/spark-metrics-20171011-224252.csv')
td811 = ((aa[' TotalDelay']/1000).sum())/60 #minutes
sd811 = (aa[' SchedulingDelay']/1000).sum()/60 #minutes
throughput811 = aa[' NumberRecords'].sum()/90  # minutes  # is that correct? I put 90 because i stopped it after 90min
pt811 = td811 - sd811
print 'Case: 8-1-1'
print 'Processing time:  ' + str(pt811)  + '   minutes'
print 'Scheduling delay: ' + str(sd811) + '  minutes'
print 'Throughput : ' + str(throughput811) + ' records/minute'
print 'Throughput : ' + str(throughput811/60) + ' records/sec'

#TODO: ADD throughput

Case: 8-1-1
Processing time:  38.8073833333   minutes
Scheduling delay: 438.892283333  minutes
Throughput : 1975 records/minute
Throughput : 32 records/sec


## 4-1-1

In [145]:
plot_throughput_and_Delays('4-1-1-1r/spark-metrics-20171011-215358.csv',4)

aa = pd.read_csv(directory+ '4-1-1-1r/spark-metrics-20171011-215358.csv')
td411 = ((aa[' TotalDelay']/1000).sum())/60 #minutes
sd411 = (aa[' SchedulingDelay']/1000).sum()/60 #minutes
throughput411 = aa[' NumberRecords'].sum()/td411  # minutes
pt411 = td411 - sd411
print 'Case: 4-1-1'
print 'Processing time:  ' + str(pt411)  + '   minutes'
print 'Scheduling delay: ' + str(sd411) + '  minutes'
print 'Throughput : ' + str(throughput411) + ' records/minute'
print 'Throughput : ' + str(throughput411/60) + ' records/sec'

#TODO: ADD throughput

/home/georgeha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:35: BokehDeprecationWarning: bokeh.io.hplot was deprecated in Bokeh 0.12.0; please use bokeh.models.layouts.Row instead


Case: 4-1-1
Processing time:  18.7446333333   minutes
Scheduling delay: 38.3706833333  minutes
Throughput : 1556.29006697 records/minute
Throughput : 25.9381677828 records/sec


## 16-1-1-1r

In [147]:
plot_throughput_and_Delays('16-1-1-1r/spark-metrics-20171012-015340.csv',4)

aa = pd.read_csv(directory+ '16-1-1-1r/spark-metrics-20171012-015340.csv')
td1611 = ((aa[' TotalDelay']/1000).sum())/60 #minutes
sd1611 = (aa[' SchedulingDelay']/1000).sum()/60 #minutes
throughput1611 = aa[' NumberRecords'].sum()/td411  # minutes
pt1611 = td1611 - sd1611
print 'Case: 16-1-1'
print 'Processing time:  ' + str(pt1611)  + '   minutes'
print 'Scheduling delay: ' + str(sd1611) + '  minutes'
print 'Throughput : ' + str(throughput1611) + ' records/minute'
print 'Throughput : ' + str(throughput1611/60) + ' records/sec'

/home/georgeha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:35: BokehDeprecationWarning: bokeh.io.hplot was deprecated in Bokeh 0.12.0; please use bokeh.models.layouts.Row instead


Case: 16-1-1
Processing time:  79.35525   minutes
Scheduling delay: 2441.66436667  minutes
Throughput : 6225.16026787 records/minute
Throughput : 103.752671131 records/sec


## 32-1-1-1r

In [156]:
plot_throughput_and_Delays('32-1-1-1r/spark-metrics-20171012-091346.csv',4)

aa = pd.read_csv(directory+ '32-1-1-1r/spark-metrics-20171012-091346.csv')
td3211 = ((aa[' TotalDelay']/1000).sum())/60 #minutes
sd3211 = (aa[' SchedulingDelay']/1000).sum()/60 #minutes
throughput3211 = aa[' NumberRecords'].sum()/td3211  # minutes
pt3211 = td3211 - sd3211
print 'Case: 32-1-1'
print 'Processing time:  ' + str(pt3211)  + '   minutes'
print 'Scheduling delay: ' + str(sd3211) + '  minutes'
print 'Throughput : ' + str(throughput3211) + ' records/minute'
print 'Throughput : ' + str(throughput3211/60) + ' records/sec'

/home/georgeha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:35: BokehDeprecationWarning: bokeh.io.hplot was deprecated in Bokeh 0.12.0; please use bokeh.models.layouts.Row instead


Case: 32-1-1
Processing time:  41.3207833333   minutes
Scheduling delay: 21.3965  minutes
Throughput : 1136.22587288 records/minute
Throughput : 18.9370978813 records/sec


## 48-1-1

In [149]:
plot_throughput_and_Delays('48-1-1/spark-metrics-20171006-122039.csv',4)

aa = pd.read_csv(directory+ '48-1-1/spark-metrics-20171006-122039.csv')
td4811 = ((aa[' TotalDelay']/1000).sum())/60 #minutes
sd4811 = (aa[' SchedulingDelay']/1000).sum()/60 #minutes
throughput4811 = aa[' NumberRecords'].sum()/td4811  # minutes
pt4811 = td4811 - sd4811
print 'Case: 48-1-1'
print 'Processing time:  ' + str(pt4811)  + '   minutes'
print 'Scheduling delay: ' + str(sd4811) + '  minutes'
print 'Throughput : ' + str(throughput4811) + ' records/minute'
print 'Throughput : ' + str(throughput4811/60) + ' records/sec'

/home/georgeha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:35: BokehDeprecationWarning: bokeh.io.hplot was deprecated in Bokeh 0.12.0; please use bokeh.models.layouts.Row instead


Case: 48-1-1
Processing time:  76.6005833333   minutes
Scheduling delay: 55.3327666667  minutes
Throughput : 1859.56772871 records/minute
Throughput : 30.9927954784 records/sec


## test decay factor

In [155]:
plot_decays('8-1-1-decay/spark-metrics-20171012-181732.csv','8-1-1 : decay=0','8-1-1-1r/spark-metrics-20171011-224252.csv','decay=1',5)

/home/georgeha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:43: BokehDeprecationWarning: bokeh.io.hplot was deprecated in Bokeh 0.12.0; please use bokeh.models.layouts.Row instead


In the experiment above I am utilizng one wrangler node to do measure the scheduling delay and total delayof spark consumer. I am running 8 concurent producers (in the above plot) <br>

 * Each producer is producing 2.5GB of data (floats)
 * Application: Spark Streaming k-means with 1 Consumer Node
 * Message size is 5K 3-D points
 ##### Observations: 
 * The delay factor appears to be the same. 
 * I am doing again this experiment because I only captured 7 batches
 * The decay factor α can be used to ignore the past: with α=1 all data will be used from the beginning; with α=0 only the most recent data will be used.

### Plot Higher Level Stats

 x-Axis: number of producers: y-axis: Processing Time (incl. error bars)
- x-Axis: number of producers: y-axis: Throughput (incl. error bars)
- x-Axis: number of producers: y-axis: Scheduling Delay (incl. error bars)

In [157]:
#1 # 4   #8  #16  #32 #48
throughputs = [4025,1556.29006697,1975,6225.16026787,1136.22587288,1859.56772871]

sched_delay = [0,38,438,2441,21,55]  

processing_time = [5,18,38,79,41,76]

x_values = [1,4,8,16,32,48]



In [162]:
## create the dataframe
myDF_5k = pd.DataFrame(dict(Production_Rate=throughputs,label=[1, 4, 8, 16,32,48]))

myDF_5k_pt = pd.DataFrame(dict(Production_Rate=processing_time,label=[1, 4, 8, 16,32,48]))

myDF_5k_sd = pd.DataFrame(dict(Production_Rate=sched_delay,label=[1, 4, 8, 16,32,48]))


In [164]:
p1 = Bar(myDF_5k,values='Production_Rate',label='label', xlabel = 'producers', ylabel="Throughput: Records/min", title='Machine: Wrangler - Nodes:1 - Throughput: Records/min')
#show(p1)
p2 = Bar(myDF_5k_pt,values='Production_Rate',label='label', xlabel = 'producers', ylabel="Processing Time: Records/min", title='Machine: Wrangler - Nodes:1 - Processing Time of Spark Consumer')
p3 = Bar(myDF_5k_sd,values='Production_Rate',label='label', xlabel = 'producers', ylabel="Scheduling Delay: Records/min", title='Machine: Wrangler - Nodes:1 - Scheduling Delay: Records/min')


# put all the plots in an HBox
plots = hplot(p1, p2,p3)
# show the results
bplt.show(plots)

/home/georgeha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: BokehDeprecationWarning: bokeh.io.hplot was deprecated in Bokeh 0.12.0; please use bokeh.models.layouts.Row instead


In [165]:
## TODO: Add the number of batches of each experiment. The experiment has a timer of 1 hour 30 minutes